## ML project - CalCOFI dataset

As explained in the README.md we are working CalCOFI dataset.

The data and information about it can be found here : https://www.kaggle.com/datasets/sohier/calcofi/

The description of the columns: https://new.data.calcofi.com/index.php/database/calcofi-database/bottle-field-descriptions.


Let's start with loading in the data and see what we haev there.

#### Loading datasets

In [ ]:
import pandas as pd
import numpy as np
import helpers

np.random.seed(helpers.random_seed)

In [ ]:
# paths to data folders
bottle_data_path = '../data/bottle'
cast_data_path = '../data/cast'

In [ ]:
# reading in
bottle_df = pd.read_csv(f'{bottle_data_path}/bottle.csv', encoding='UTF-8')
cast_df = pd.read_csv(f'{cast_data_path}/cast.csv', encoding='UTF-8')

# Initial exploration of dataset
As mentioned before, CalCOFI dataset contains two sets of data, one (bottle) with oceanographic data, and the other (cast) with sampling stations data and geolocation data. Let's look at the samples of each set.

In [ ]:
bottle_df.sample(5)

In [ ]:
cast_df.sample(5)

We will mostly deal with Bottle table due to it containing actual water quality data. 

Cast table, on the other hand, might be used if we would like to retrieve the geolocation of the measurements. So let's drop everything not related to the geolocation from Cast table and merge this with bottle dataset.

In [ ]:
cast_df = cast_df[['Cst_Cnt', 'Lat_Dec', 'Lon_Dec']]

df = pd.merge(bottle_df, cast_df, on='Cst_Cnt')
df.shape

We can now make plot of location of sampling stations (boats) from cast dataset, and we can clearly see the American west coast landline.

In [ ]:
cast_df.plot(kind="scatter", x="Lon_Dec", y="Lat_Dec", grid=True, xlabel='Longitude', ylabel='Latitude')

#### Dataset general info

In [ ]:
bottle_df_info = df.info()

As we can see, some variables have a lot of missing values.

In [ ]:
# the size of a current dataset
df.shape

#### What problem are we solving?

The idea of the project will be to predict the temperature of water.

### Removing excessive data

#### RecInd column

According to the description, this is a categorical column with values:

**Record Indicator**
* "3" - Observed Data
* "4" - Educated office guess (ghost)
* "5" - Data from STD or CTD device
* "6" - Duplicate Depth
* "7" - Interpolated to a standard depth

In the context of the project, We suggest omitting interpolated data, as it is generally used to compare observations in different regions of the world, and for us actual observations have more value. Also, as it seems, "Duplicate Depth" means extra measurement taken, so, perhaps, we can omit it as well. Therefore, we'll keep only indicators "3" and "5", which comes with a drawback of reducing the amount of rows almoust by half.

In [ ]:
df.value_counts('RecInd').to_frame()

In [ ]:
bottle_drop_idx = df.loc[(df['RecInd'] != 3) & (df['RecInd'] != 5)].index

df.drop(index=bottle_drop_idx, inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df.info()

#### "Reported" columns

Some columns have "reported" counterparts. Perhaps, they are highly correlated or even very close to each other, and we can remove even more columns. Let's calculate difference between a pair of columns divided by mean value of the reported column.

In [ ]:
reported_cols_pairs = [
    ('Depthm', 'R_Depth'), 
    ('T_degC', 'R_TEMP'),
    ('R_POTEMP', 'R_TEMP'), 
    ('Salnty', 'R_SALINITY'), 
    ('STheta', 'R_SIGMA'),
    ('O2ml_L', 'R_O2'), 
    ('O2Sat', 'R_O2Sat'),
    ('SiO3uM', 'R_SIO3'),
    ('PO4uM', 'R_PO4'),
    ('NO3uM', 'R_NO3'),
    ('NO2uM', 'R_NO2'),
    ('NH3uM', 'R_NH4'),
    ('ChlorA', 'R_CHLA'),
    ('Phaeop', 'R_PHAEO')
]

reported_diff_mean = pd.Series()

for pair in reported_cols_pairs:
    pair_key = f'{pair[0]}|{pair[1]}'
    reported_diff_mean[pair_key] = np.nanmean(df[pair[0]] - df[pair[1]]) / np.nanmean(df[pair[1]])

reported_diff_mean

We see that the values are effectively negligible, hence we can keep only "reported" columns.

In [ ]:
df.drop(columns=[c[0] for c in reported_cols_pairs], inplace=True)
df.shape

#### Primary Productivity

Unfortunately, there is too little data about primary productivity measurements (the rate at which energy is converted to organic substances by photosynthetic producers), which could otherwise be very useful. Perhaps, it could be studied separately another time. CalCOFI uses radioactive carbon $C_{14}$ for primary productivity measurements (as far as I understand, carbon is injected in the ecosystem and then "travels" together with microorganisms), therefore we drop columns related to it.

In [ ]:
carbon_cols = ['C14As1', 'C14A1p', 'C14A1q', 'C14As2', 'C14A2p', 'C14A2q', 'DarkAs', 'DarkAp', 'DarkAq', 'MeanAs', 'MeanAp', 'MeanAq', 'IncTim', 'LightP']

df.drop(columns=carbon_cols, inplace=True)
df.shape

#### Precision columns
There are some measurement precision columns present in the dataset. We don't need them for the model construction. 

In [ ]:
prec_cols = ['T_prec', 'S_prec']
df.drop(columns=prec_cols, inplace=True)
df.shape

#### Quality Codes columns

According to https://new.data.calcofi.com/index.php/database/calcofi-database/bottle-field-descriptions, when we have quality code 0 or blank, it means that the sample has good quality. These are categorical columns.

**Quality Code**

* Blank - Data OK
* "4" - Zeroed due to value below detection limit
* "6" - Data taken from CTD device
* "8" - Originator thinks value is suspect
* "9" - Missing Data

My plan here is to remove these columns completely, but first it would be useful to know whether the values in meaningful columns are not filled because they are below detection level or just missing.

In [ ]:
col_qual_pairs = [
    ('R_TEMP', 'T_qual'),
    ('R_SALINITY', 'S_qual'),
    ('R_PRES', 'P_qual'),
    ('R_O2', 'O_qual'),
    ('R_SIGMA', 'SThtaq'),
    ('R_O2Sat', 'O2Satq'),
    ('R_CHLA', 'Chlqua'),
    ('R_PHAEO', 'Phaqua'),
    ('R_PO4', 'PO4q'),
    ('R_SIO3', 'SiO3qu'),
    ('R_NO2', 'NO2q'),
    ('R_NO3', 'NO3q'),
    ('R_NH4', 'NH3q')
]

In [ ]:
for (col, qual) in col_qual_pairs:
    qual_values = df[[col, qual]].value_counts(qual)
    print(qual_values)

For each column with quality=8, we are replacing its value with NaN.

In [ ]:
for (col, qual) in col_qual_pairs:
    df.loc[df[qual] == 8, col] = np.nan

    print(df.loc[(df[qual] == 8) & df[col].notnull()].shape)

It appears, there are indeed a lot of missing values in columns, according to the authors of the dataset. We should consider it while imputing missing data. For now proceed to dropping quality columns.

In [ ]:
qual_cols = df.filter(regex='.*_qual|.*qua|.*q').columns

df.drop(columns=qual_cols, inplace=True)
df.shape

#### "Descriptive" and other data

For the modelling purposes we do not need the columns like bottle count, different IDs etc., so we drop them.

In [ ]:
almost_empty_cols = ['Cst_Cnt', 'Btl_Cnt', 'Sta_ID', 'Depth_ID', 'BtlNum', 'RecInd', 'R_SAMP']

df.drop(columns=almost_empty_cols, inplace=True)
df.shape

#### Almost empty columns

Remove columns with little to no data.

In [ ]:
almost_empty_cols = ['DIC1', 'DIC2', 'TA1', 'TA2', 'pH2', 'pH1', 'DIC Quality Comment']

df.drop(columns=almost_empty_cols, inplace=True)
df.shape

#### Oxy_µmol/Kg

This columns seems to have similar meaning to R_O2, just in different measurement units. Can we remove it?

In [ ]:
df[['Oxy_µmol/Kg', 'R_O2']].corr()

They are indeed completely correlated, therefore the answer is yes, we can drop Oxy_µmol/Kg.

In [ ]:
df.drop(columns=['Oxy_µmol/Kg'], inplace=True)
df.shape

In [ ]:
df.info()

## R_TEMP - target feature

Since we decided to use R_TEMP column as a feature to probe against, imputing missing values is not a good strategy. Therefore we have to keep only the rows with R_TEMP present.

In [ ]:
df.drop(df[df[helpers.target_feature].isnull()].index, inplace=True)
df.shape

### Saving the data

To conclude this part, we managed to greatly reduce the amount of features. Some of rows have also been removed due to their supposed syntheticity in the original dataset (interpolation). Now we can proceed to train/test split and to the exploratory data analysis to prepare the data for modelling.

In [ ]:
df.to_csv(f'../data/dataset.csv')